# Goal : To predict whether a loan will be paid off on time or not










## Notebook structure

###  Data Cleaning
Prepare data on loans for predictive modeling.

### Preparing The Features
Exploring and preparing the features for modeling.

### Making Predictions
Train, test, and iterate on machine learning models.

## Import libraries

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_predict, KFold


## Reducing the size of LoanStats3a.csv by:

1. Removing the first line: because it contains the extraneous text Notes offered by Prospectus (https://www.lendingclub.com/info/prospectus.action) instead of the column titles, which prevents the dataset from being parsed by the pandas library properly
2. Removing the desc column:which contains a long text explanation for each loan
3. Removing the url column:which contains a link to each loan on Lending Club which can only be accessed with an investor account
4. Removing all columns containing more than 50% missing values:which allows us to move faster since we can spend less time trying to fill these values

In [12]:
loans_2007 = pd.read_csv('LoanStats3a.csv', skiprows=1)
half_count = len(loans_2007) / 2
loans_2007 = loans_2007.dropna(thresh=half_count, axis=1)
loans_2007 = loans_2007.drop(['desc', 'url'],axis=1)
loans_2007.to_csv('loans_2007.csv', index=False)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2705: DtypeWarning: Columns (0,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Read into Pandas Dataframe

In [13]:

loans_2007 = pd.read_csv("loans_2007.csv")
loans_2007.drop_duplicates()
print(loans_2007.iloc[0])
print(loans_2007.shape[1])

/home/ubuntu/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2705: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


id                                1077501
member_id                      1.2966e+06
loan_amnt                            5000
funded_amnt                          5000
funded_amnt_inv                      4975
term                            36 months
int_rate                           10.65%
installment                        162.87
grade                                   B
sub_grade                              B2
emp_title                             NaN
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
issue_d                          Dec-2011
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
zip_code                            860xx
addr_state                             AZ
dti                                 27.65
delinq_2yrs                       

### After analyzing each column, we can conclude that the following features need to be removed:

1. id: randomly generated field by Lending Club for unique identification purposes only
2. member_id: also a randomly generated field by Lending Club for unique identification purposes only
3. funded_amnt: leaks data from the future (after the loan is already started to be funded)
4. funded_amnt_inv: also leaks data from the future (after the loan is already started to be funded)
5. grade: contains redundant information as the interest rate column (int_rate)
6. sub_grade: also contains redundant information as the interest rate column (int_rate)
7. emp_title: requires other data and a lot of processing to potentially be useful
8. issue_d: leaks data from the future (after the loan is already completed funded)


#### Lending Club assigns a grade and a sub-grade based on the borrower's interest rate. While the grade and sub_grade values are categorical, the int_rate column contains continuous values, which are better suited for machine learning.

## First group of features ##

In [14]:

loans_2007 = loans_2007.drop(["id", "member_id", "funded_amnt", "funded_amnt_inv", "grade", "sub_grade", "emp_title", "issue_d"], axis=1)



## Second group of features ##

### Within this group of columns, we need to drop the following columns:

1. zip_code: redundant with the addr_state column since only the first 3 digits of the 5 digit zip code are visible (which only can be used to identify the state the borrower lives in)
2. out_prncp: leaks data from the future, (after the loan already started to be paid off)
3. out_prncp_inv: also leaks data from the future, (after the loan already started to be paid off)
4. total_pymnt: also leaks data from the future, (after the loan already started to be paid off)
5. total_pymnt_inv: also leaks data from the future, (after the loan already started to be paid off)
6. total_rec_prncp: also leaks data from the future, (after the loan already started to be paid off)


#### The out_prncp and out_prncp_inv both describe the outstanding principal amount for a loan, which is the remaining  amount the borrower still owes. These 2 columns as well as the total_pymnt column describe properties of the loan after it's fully funded and started to be paid off. This information isn't available to an investor before the loan is fully funded and we don't want to include it in our model.


In [15]:
loans_2007 = loans_2007.drop(["zip_code", "out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv", "total_rec_prncp"], axis=1)




## Third group of features ##


### All of these columns in this group leak data from the future, meaning that they're describing aspects of the loan after it's already been fully funded and started to be paid off by the borrower, we need to drop the following columns:

1. total_rec_int: leaks data from the future, (after the loan already started to be paid off),
2. total_rec_late_fee: also leaks data from the future, (after the loan already started to be paid off),
3. recoveries: also leaks data from the future, (after the loan already started to be paid off),
4. collection_recovery_fee: also leaks data from the future, (after the loan already started to be paid off),
5. last_pymnt_d: also leaks data from the future, (after the loan already started to be paid off),
6. last_pymnt_amnt: also leaks data from the future, (after the loan already started to be paid off).


In [16]:
loans_2007 = loans_2007.drop(["total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt"], axis=1)
print(loans_2007.iloc[0])
print(loans_2007.shape[1])

loan_amnt                            5000
term                            36 months
int_rate                           10.65%
installment                        162.87
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
addr_state                             AZ
dti                                 27.65
delinq_2yrs                             0
earliest_cr_line                 Jan-1985
inq_last_6mths                          1
open_acc                                3
pub_rec                                 0
revol_bal                           13648
revol_util                          83.7%
total_acc                               9
initial_list_status                     f
last_credit_pull_d               S

## Target column ##

In [17]:
print(loans_2007['loan_status'].value_counts())

Fully Paid                                             33586
Charged Off                                             5653
Does not meet the credit policy. Status:Fully Paid      1988
Does not meet the credit policy. Status:Charged Off      761
Current                                                  513
In Grace Period                                           16
Late (31-120 days)                                        12
Late (16-30 days)                                          5
Default                                                    1
Name: loan_status, dtype: int64


## Binary classification ##

In [18]:

loans_2007 = loans_2007[(loans_2007['loan_status'] == "Fully Paid") | (loans_2007['loan_status'] == "Charged Off")]

status_replace = {
    "loan_status" : {
        "Fully Paid": 1,
        "Charged Off": 0,
    }
}

loans_2007 = loans_2007.replace(status_replace)


## Removing single value columns ##

In [19]:
orig_columns = loans_2007.columns
drop_columns = []
for col in orig_columns:
    col_series = loans_2007[col].dropna().unique()
    if len(col_series) == 1:
        drop_columns.append(col)
loans_2007 = loans_2007.drop(drop_columns, axis=1)
print(drop_columns)

['initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


In [21]:
loans_2007.to_csv("filtered_loans_2007.csv")

# Preparing features

In [22]:

loans = pd.read_csv('filtered_loans_2007.csv')
null_counts = loans.isnull().sum()
print(null_counts)

Unnamed: 0                0
loan_amnt                 0
term                      0
int_rate                  0
installment               0
emp_length                0
home_ownership            0
annual_inc                0
verification_status       0
loan_status               0
pymnt_plan                0
purpose                   0
title                    10
addr_state                0
dti                       0
delinq_2yrs               0
earliest_cr_line          0
inq_last_6mths            0
open_acc                  0
pub_rec                   0
revol_bal                 0
revol_util               50
total_acc                 0
last_credit_pull_d        2
pub_rec_bankruptcies    697
dtype: int64


## Handling missing values ##

In [23]:
loans = loans.drop("pub_rec_bankruptcies", axis=1)
loans = loans.dropna(axis=0)
print(loans.dtypes.value_counts())

object     12
float64    10
int64       2
dtype: int64



## Text columns ##


In [24]:
object_columns_df = loans.select_dtypes(include=["object"])
print(object_columns_df.iloc[0])

term                     36 months
int_rate                    10.65%
emp_length               10+ years
home_ownership                RENT
verification_status       Verified
pymnt_plan                       n
purpose                credit_card
title                     Computer
addr_state                      AZ
earliest_cr_line          Jan-1985
revol_util                   83.7%
last_credit_pull_d        Sep-2016
Name: 0, dtype: object



##  First 5 categorical columns ##

In [25]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']
for c in cols:
    print(loans[c].value_counts())

RENT        18677
MORTGAGE    17381
OWN          3020
OTHER          96
NONE            3
Name: home_ownership, dtype: int64
Not Verified       16809
Verified           12515
Source Verified     9853
Name: verification_status, dtype: int64
10+ years    8715
< 1 year     4542
2 years      4344
3 years      4050
4 years      3385
5 years      3243
1 year       3207
6 years      2198
7 years      1738
8 years      1457
9 years      1245
n/a          1053
Name: emp_length, dtype: int64
 36 months    29041
 60 months    10136
Name: term, dtype: int64
CA    7019
NY    3757
FL    2831
TX    2693
NJ    1825
IL    1513
PA    1493
VA    1388
GA    1381
MA    1322
OH    1197
MD    1039
AZ     863
WA     830
CO     777
NC     772
CT     738
MI     718
MO     677
MN     608
NV     488
SC     469
WI     447
AL     441
OR     441
LA     432
KY     319
OK     294
KS     264
UT     255
AR     241
DC     211
RI     197
NM     187
WV     174
HI     170
NH     169
DE     113
MT      84
WY      83
AK      

In [26]:
print(loans["purpose"].value_counts())
print(loans["title"].value_counts())

debt_consolidation    18355
credit_card            5073
other                  3921
home_improvement       2944
major_purchase         2178
small_business         1792
car                    1534
wedding                 940
medical                 688
moving                  580
vacation                377
house                   372
educational             320
renewable_energy        103
Name: purpose, dtype: int64
Debt Consolidation                         2142
Debt Consolidation Loan                    1670
Personal Loan                               650
Consolidation                               501
debt consolidation                          495
Credit Card Consolidation                   354
Home Improvement                            350
Debt consolidation                          331
Small Business Loan                         317
Credit Card Loan                            310
Personal                                    306
Consolidation Loan                          255
Home

## Categorical columns ##

In [27]:
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}
loans = loans.drop(["last_credit_pull_d", "earliest_cr_line", "addr_state", "title"], axis=1)
loans["int_rate"] = loans["int_rate"].str.rstrip("%").astype("float")
loans["revol_util"] = loans["revol_util"].str.rstrip("%").astype("float")
loans = loans.replace(mapping_dict)


## Dummy variables ##

In [28]:
cat_columns = ["home_ownership", "verification_status", "purpose", "term"]
dummy_df = pd.get_dummies(loans[cat_columns])
loans = pd.concat([loans, dummy_df], axis=1)
loans = loans.drop(cat_columns, axis=1)

In [29]:
loans.to_csv("cleaned_loans_2007.csv")

# Making Predictions

In [35]:
loans = pd.read_csv("cleaned_loans_2007.csv")
loans = loans.drop(["Unnamed: 0","Unnamed: 0.1","pymnt_plan" ], axis=1)
print(loans.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39177 entries, 0 to 39176
Data columns (total 38 columns):
loan_amnt                              39177 non-null float64
int_rate                               39177 non-null float64
installment                            39177 non-null float64
emp_length                             39177 non-null int64
annual_inc                             39177 non-null float64
loan_status                            39177 non-null int64
dti                                    39177 non-null float64
delinq_2yrs                            39177 non-null float64
inq_last_6mths                         39177 non-null float64
open_acc                               39177 non-null float64
pub_rec                                39177 non-null float64
revol_bal                              39177 non-null float64
revol_util                             39177 non-null float64
total_acc                              39177 non-null float64
home_ownership_MORTGAGE    

In [41]:
loans.describe()

,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,...,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
count,39177.000000,39177.000000,39177.000000,39177.000000,3.917700e+04,39177.000000,39177.000000,39177.000000,39177.000000,39177.000000,...,39177.000000,39177.000000,39177.000000,39177.000000,39177.000000,39177.000000,39177.000000,39177.000000,39177.000000,39177.000000
mean,11143.689537,11.976177,323.514635,4.828471,6.891654e+04,0.856191,13.298736,0.146540,0.870230,9.292646,...,0.055594,0.017561,0.014805,0.100084,0.002629,0.045741,0.009623,0.023994,0.741277,0.258723
std,7398.202266,3.706860,208.483501,3.603729,6.400410e+04,0.350900,6.674513,0.491341,1.071069,4.400309,...,0.229139,0.131352,0.120772,0.300116,0.051208,0.208926,0.097625,0.153031,0.437939,0.437939
min,500.000000,5.420000,15.690000,0.000000,4.000000e+03,0.000000,0.000000,0.000000,0.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5425.000000,8.940000,166.500000,2.000000,4.020000e+04,1.000000,8.160000,0.000000,0.000000,6.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,10000.000000,11.830000,279.160000,4.000000,5.900000e+04,1.000000,13.390000,0.000000,1.000000,9.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,15000.000000,14.460000,428.030000,9.000000,8.200000e+04,1.000000,18.580000,0.000000,1.000000,12.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,35000.000000,24.590000,1305.190000,10.000000,6.000000e+06,1.000000,29.990000,11.000000,8.000000,44.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Picking Error metrics

###  Our objective in this is to make money -- we want to fund enough loans that are paid off on time to offset our losses from loans that aren't paid off. An error metric will help us determine if our algorithm will make us money or lose us money.


###  In this case, we're primarily concerned with false positives and false negatives. Both of these are different types of misclassifications. With a false positive, we predict that a loan will be paid off on time, but it actually isn't. This costs us money, since we fund loans that lose us money. With a false negative, we predict that a loan won't be paid off on time, but it actually would be paid off on time. This loses us potential money, since we didn't fund a loan that actually would have been paid off.

In [99]:
def error_metrics(predictions):
    predictions_dataframe=pd.DataFrame(predictions)

    # False positives.
    fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
    fp = len(predictions_dataframe[fp_filter])

    # True positives.
    tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
    tp = len(predictions_dataframe[tp_filter])

    # False negatives.
    fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
    fn = len(predictions_dataframe[fn_filter])

    # True negatives
    tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
    tn = len(predictions_dataframe[tn_filter])

    # Rates
    tpr = float(tp) /float(tp + fn)
    fpr = float(fp) / float(fp + tn)
    print(tp,fp,tn,fn)
    print(tpr)
    print(fpr)


# Logistic Regression ##

In [98]:

lr = LogisticRegression()
cols = loans.columns
train_cols = cols.drop("loan_status")
features = loans[train_cols]
target = loans["loan_status"]
lr.fit(features, target)
predictions = lr.predict(features)
error_metrics(predictions)

(33507, 5618, 16, 36)
0.998926750738
0.997160099397



## Cross Validation 


###  Cross validation splits the dataset into groups, then makes predictions on each group using the other groups as training data. This ensures that we don't overfit by generating predictions on the same data that we train our algorithm with.

### We'll create an instance of KFold, which will perform 3 fold cross validation across our dataset. We set random_state to 1 to ensure that the folds are always consistent, and we can compare scores between runs. If we don't, each fold will be randomized every time, making it hard to tell if we're improving our model or not.



### If we pass the instance of KFold into cross_val_predict, it will then perform 3 fold cross validation to generate unbiased predictions.



### Once we have cross validated predictions, we can compute true positive rate and false positive rate.

In [97]:

lr = LogisticRegression()
kf = KFold(features.shape[0], random_state=1)
predictions = cross_val_predict(lr, features, target, cv=kf)
predictions = pd.Series(predictions)
error_metrics(predictions)

(33512, 5618, 16, 31)
0.999075813135
0.997160099397


## Penalizing the classifier ##

### The penalty means that the logistic regression classifier pays more attention to correctly classifying rows where loan_status is 0. This lowers accuracy when loan_status is 1, but raises accuracy when loan_status is 0.



### By setting the class_weight parameter to balanced, the penalty is set to be inversely proportional to the class frequencies. You can read more about the parameter here. This would mean that for the classifier, correctly classifying a row where loan_status is 0 is 6 times more important than correctly classifying a row where loan_status is 1.

In [96]:
lr = LogisticRegression(class_weight="balanced")
kf = KFold(features.shape[0], random_state=1)
predictions = cross_val_predict(lr, features, target, cv=kf)
predictions = pd.Series(predictions)
error_metrics(predictions)


(22266, 2218, 3416, 11277)
0.663804668634
0.393681221157


## Manual penalties ##

### We significantly improved false positive rate in the last model by balancing the classes, which reduced true positive rate. Our true positive rate is now around 67%, and our false positive rate is around 40%. From a conservative investor's standpoint, it's reassuring that the false positive rate is lower because it means that we'll be able to do a better job at avoiding bad loans than if we funded everything. However, we'd only ever decide to fund 67% of the total loans (true positive rate), so we'd immediately reject a good amount of loans.








### In the last model, the penalty scikit-learn imposed for misclassifying a 0 would have been around 5.89 (since there are 5.89 times as many 1s as 0s).We can try to lower the false positive rate further by assigning a harsher penalty for misclassifying the negative class. While setting class_weight to balanced will automatically set a penalty based on the number of 1s and 0s in the column, we can also set a manual penalty. 

In [95]:
penalty = {
    0: 10,
    1: 1
}
# The above dictionary will impose a penalty of 10 for misclassifying a 0, and a penalty of 1 for misclassifying a 1
lr = LogisticRegression(class_weight=penalty)
kf = KFold(features.shape[0], random_state=1)
predictions = cross_val_predict(lr, features, target, cv=kf)
predictions = pd.Series(predictions)
error_metrics(predictions)


(7879, 476, 5158, 25664)
0.234892526011
0.0844870429535


###  Assigning manual penalties lowered the false positive rate to 7%, and thus lowered our risk. But this comes at the expense of true positive rate. While we have fewer false positives, we're also missing opportunities to fund more loans and potentially make more money. For a conservative investor, this strategy makes sense, but it's worth keeping in mind the tradeoffs.

## Random forests ##

In [94]:
rf = RandomForestClassifier(class_weight="balanced", random_state=1)
kf = KFold(features.shape[0], random_state=1)
predictions = cross_val_predict(rf, features, target, cv=kf)
predictions = pd.Series(predictions)
error_metrics(predictions)




(32658, 5312, 322, 885)
0.973615955639
0.942847000355


In [102]:
penalty = {
    0: 10,
    1: 1
}
# The above dictionary will impose a penalty of 10 for misclassifying a 0, and a penalty of 1 for misclassifying a 1
rf = RandomForestClassifier(class_weight=penalty,random_state=1)
kf = KFold(features.shape[0], random_state=1)
predictions = cross_val_predict(rf, features, target, cv=kf)
predictions = pd.Series(predictions)
error_metrics(predictions)

(32810, 5317, 317, 733)
0.978147452524
0.943734469294


### Ultimately, our best model had a false positive rate of 8.4%, and a true positive rate of 23.4%. For a conservative investor, this means that they make money as long as the interest rate is high enough to offset the losses from 7% of borrowers defaulting, and that the pool of 23.4% of borrowers is large enough to make enough interest money to offset the losses.



### If we had randomly picked loans to fund, borrowers would have defaulted on 14.5% of them, and our model is better than that, although we're excluding more loans than a random strategy would. 